In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
import json # library to handle JSON files


import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import urllib.request


import time
from bs4 import BeautifulSoup

import re
print("imported")

imported


In [5]:
#getting data
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(page.text, 'html.parser')

#variables for table, table_header, rows and data etc
the_table = soup.find('table')
table_header = the_table.findAll('th')
rows = the_table.findAll('tr')
data = the_table.findAll('td')
 
#removing html tags from headers    
for i, head in enumerate(table_header): table_header[i]=str(table_header[i]).replace("<th>","").replace("</th>","").replace("\n","")

#removing html tags from rows   
for i, row in enumerate(rows): rows[i] = str(rows[i]).replace("\n</td></tr>","").replace("<tr>\n<td>","")

# creating dataframe
df=pd.DataFrame(rows)

# make dataframe, expand rows and drop the old one:
df[table_header] = df[0].str.split("</td>\n<td>", 2, expand = True) 
df.drop(columns=[0],inplace=True)

#rename column name 
df.rename(columns={'Postal Code':'PostalCode'},inplace = True)

#droping the first row
df = df.drop(0)

#filtering out table--removing not assigned rows
df=df[df['Borough'] != 'Not assigned\n']

# remove \n tag from data
df.update(
    df.Borough.loc[
        lambda x: x.str.contains('\n')
    ].str.replace("\n",""))


df.update(
    df.PostalCode.loc[
        lambda x: x.str.contains('\n')
    ].str.replace("\n",""))

# replace "not assigned" Nieghborhood with data from Borogh
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)

df.head(10)




,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,"Malvern, Rouge"
12,M3B,North York,Don Mills
13,M4B,East York,"Parkview Hill, Woodbine Gardens"
14,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
#add Geo-spatial data
df_geoData= pd.read_csv("http://cocl.us/Geospatial_data")
df_geoData.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_geoData.head
df_geoData.set_index("Postcode")

#rename column as that of geodta to make a join
df.rename(columns={'PostalCode':'Postcode'}, inplace=True)
df.set_index("Postcode")

#merge geospatial data, nieghbourhood data 
toronto_data=pd.merge(df,df_geoData, on='Postcode', how='inner')
toronto_data


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
